# Clustering with Spark - Data Exploration
This exercise deals with Apache Spark SQL which is the data wrangling tool of Spark and a bit similar to Pandas. As datasets we will use bills from a fictional supermarket.

To use Spark we need to **create a SparkSession**, which is the entry point to Spark SQL. Hence, **import** the class **SparkSession** from the module **pyspark.sql**. Afterwards, we create an object of that class called **spark** by using the builder of the SparkSession.

In [ ]:
# Import SparkSession
from pyspark.sql import <FILL-IN>

In [ ]:
# Create spark object
spark = SparkSession.builder.getOrCreate()

We can create a DataFrame by using the DataFrameReader read of the **spark** object. Check out the DataFrameReader to see what kind of data we can load.  

**Hint**: You can inspect all attributes and methods of an object by using the built-in **dir(moduleName/Object)** function.

Our dataset has been saved in a semistructured json format. You can load the data by using the method **spark.read.json(pathToFile)**. Please load the data **bills-1000000.json** from the data folder and call the dataframe bills.

In [ ]:
# check the dataframe reader (optional)

In [ ]:
# Create DataFrame bills
bills = spark.read.json(<FILL-IN>)

In Pandas we have encountered the dataframe methods head, info and describe. The equivalents of spark sql are **show**, **printSchema** and **describe**. Please, use the methods on the dataframe. Afterwards, answer the following questions:

1. How many columns does the dataframe hold?
2. What are the datatypes of the columns?

In [ ]:
# show


In [ ]:
# schema


In [ ]:
# describe and show


In contrast to Pandas, **Spark DataFrames are immutable objects**. You can perform transformations on a dataframe which result into a new dataframe and which do not modify the original one. For instance adding a new column yields a new dataframe.

You can select different columns by using the method **select** on a dataframe. Please, **select** the columns **customerId**, **products** and **total** of the bills DataFrame and call the resulting dataframe again **bills**. As the argument of the method you can either use a list our just comma separated column names. To check if the selection worked use the method **show**.

**Remark**: Every method which results into a new dataframe is called a transformation. **Transformations in Spark are lazy** and therefore are not really evaluated after the execution of such a statement. All the transformations are only added to the directed acyclic execution graph. The computation is **triggered when you apply an action**, like show or count. Do you remember why this behaviour is useful? And what are the drawbacks?   

In [ ]:
# select customerId, products and total
bills = bills.select(<FILL-IN>)
bills.show()

Maybe  you have noticed that the first row contains only null values. Hence, we can drop that column by using the method **dropna()** on the dataframe. Call the resulting dataframe again **bills**. Always check the resulting dataframe by using the **show** method.

**Remark**: Similar to Pandas we could have used **chaining**, i.e. using several methods in one single statement.

In [ ]:
# Drop null values
bills = bills.<FILL-IN>
bills.<FILL-IN>

The products column contains a nested array. To see the whole array you can select the **products** column by using the **select** method. Afterwards, apply the **show** method and set the arguments **n to 5** and **truncate to False**. Here, one nested array contains all the groceries that one client has bought during a single shopping.

In [ ]:
# Show complete products array
bills.select(<FILL-IN>).show(<FILL-IN>)

Next, we want to **add a new index column** called **shoppingId** which is made of a monotonically increasing integer number. Therefore, we use the function **monotonically_increasing_id** of the **function module of pyspark.sql**. Hence, **import** the **functions** module of pyspark.sql **as F**.

To add a column we can use the method **withColumn** on the Spark DataFrame. As the two arguments use **'shoppingId'** and **F.monotonically_indreasing_id()**. Call the resulting dataframe **bills**. Finally, use the show method on the DataFrame to see if everything has worked properly.

In [ ]:
# Import functions as F
from pyspark.sql import <FILL-IN> as F

In [ ]:
# Add a new column shoppingId
bills = bills.withColumn(<FILL-IN>, F.<FILL-IN>)
bills.<FILL-IN>

As in Pandas you can use a lot of different methods on a Spark DataFrame. For more functions check out the documentation:
<a href="http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame">http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame</a>. Usually you use these functions together with the method **withColumn** to add the result of the function to a dataframe.

In Spark you can also easily write your own functions, called UDFs. Let me demonstrate to you how you can construct a simple UDF:

In [ ]:
# Read and execute the UDF Demo

from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf

# Create UDF
@udf(returnType=ArrayType(StringType()))
def split_string(my_string):
    return my_string.split('-')

# Above we have used a python decorator. It is similar to:
# split_string = F.udf(f=split_string,returnType=ArrayType(StringType()))

# Not a really useful udf, but maybe you saw how easy it is :)
bills.withColumn('custom_udf', split_string('customerId')).show()

Please, use the **filter** and **count** methods to find out how many **shopping bags** were more expensive than 70 euros.

**Hint**: The method filter expects a condition and returns a dataframe. You can directly apply the count method via chaining. Use the *Shift + Tab* method to find out how you have to specify the condition.

In [ ]:
# Filter and count
bills.<FILL-IN>(bills.total > <FILL-IN>).count()

Next, we want to **create a histogram** of the column **total**. To create plots one usually transforms spark dataframes to **pandas dataframes** and uses the **matplotlib** or **seaborn** package. However, it is important to know that for a pandas dataframe the **whole data must fit into the memory of a single machine**. Therefore, it is common to aggregate the data with spark in order to transform less data to a pandas dataframe.

For this reason, we create bins of the column total by ourself. This can be achieved by using the **Bucketizer** class of the pyspark ML library. But first, please find out the **minimum** and **maximum** of the column **total**. Afterwards, use **numpy** to create an array of **evenly spaced numbers** over the min to max interval. The list should contain 10 elements.

**Hint**: Use the function linspace of numpy.

In [ ]:
# summary statistics to get min and max of total
bills.select(<FILL-IN>).describe().<FILL-IN>

In [ ]:
# Create bin array
import numpy as np
bins = np.<FILL-IN>
bins

Now, we import the Bucketizer class from the module pyspark.ml.feature.

In [ ]:
# Import Bucketizer
from pyspark.ml.feature import Bucketizer

Please, create an object **bucketizer** of that class **Bucketizer** and use as the argument *splits* the previously created *bins* array, as inputCol the column we want to bucketize and as the outputCol the string "total_buckets".

In [ ]:
# Create bucketizer
bucketizer = Bucketizer(splits=bins, inputCol="total", outputCol="total_buckets")

This kind of object is very similar to sklearn preprocessing objects (e.g. the Standardizer). It has the method **transform** which expects a dataframe as the argument and returns a transformed dataframe. Please, use that method on the **bills** dataframe. Call the resulting dataframe **total_buck** and **show** the result.

In [ ]:
# Use the transform method of the bucketizer
total_buck = bucketizer.<FILL-IN>
total_buck.show()

Now, we want to **count** the occurences of **shoppings** with respect to the **total_buckets**. Therefore, we can use the **groupBy** method on the total_buck dataframe. Afterwards, we can apply an aggregation which in this case here is **count**. Call the resulting dataframe **total_buck_counts**. As always, show the dataframe.

In [ ]:
# Perform groupBy aggregation
total_buck_counts = total_buck.<FILL-IN>('total_buckets').<FILL-IN>
total_buck_counts.<FILL-IN>

This dataframe is now much smaller than the bills dataframe. Hence, we can easily transform it to a pandas. Therefore, please use the method **toPandas()** on the total_buck_counts dataframe. Call the resulting dataframe **total_buck_counts_pandas**. How much memory does the pandas dataframe occupy?

In [ ]:
# Create pandas dataframe
total_buck_counts_pandas = total_buck_counts.<FILL-IN>

In [ ]:
# show head of pandas df
<FILL-IN>

In [ ]:
# memory usage?
<FILL-IN>

Import **seaborn** as sns, issue the command **%matplotlib inline** and use the seaborn method **barplot** to plot the histogram. **Describe the histogram**.

In [ ]:
# Import seaborn
import seaborn as sns
%matplotlib inline

In [ ]:
# Create barplot
sns.barplot(x='total_buckets', y='count', data=<FILL-IN>)

There are also other ways to plot histograms in Spark. However, they all have in common that you need all the data on a single machine.

**Remark**: Do never collect the whole dataframe to the driver, i.e. to a single JVM on a single machine. Always aggregate the data first. In Big Data the data is usually just too big to fit into the memory of one machine.

Next, we want to **explode** the **products** array of the bills dataframe. This means that each element of the array will lead to a new row in the dataframe. Therefore, pyspark provides a function called explode which is part of the functions module which we have already imported.

Please use the method **withColumn** on the dataframe bills. As the argument use **'products'** and the explode function of the functions module which we have imported as F. To the explode function please pass the **products column**.
Call the resulting dataframe **bills_exploded** and use the show method to see if the transformation has worked as expected.

In [ ]:
# Explode the column products of the bills dataframe
<FILL-IN> = bills.withColumn(<FILL-IN>, F.<FILL-IN>('products'))
bills_exploded.show<FILL-IN>

Checkout out the schema of the dataframe by using the printSchema method.

In [ ]:
# print Schema
bills_exploded.<FILL-IN>

To select elements of an *struct* (named array) datatype you can use dot notation, e.g. 'products.name'. Please try to select only the price column of the dataframe bills_exploded.

In [ ]:
# Select element of a struct datatype
bills_exploded.select(<FILL-IN>).show()

We want to extract all the elements inside the products column. This can be done by using the **asterisk symbol \***. Please, select **shoppingId**, **customerId**, **total** and all attributes of **products**. Afterwards, **drop** the column **unit** by using the method **drop()**. Call the resulting dataframe **bills_exploded**.

In [ ]:
# Extract elements of the struct column products
<FILL-IN> = bills_exploded.select("<FILL-IN>*",<FILL-IN>, <FILL_IN>, 'total').drop(<FILL-IN>)
bills_exploded.show()

In contrast to Pandas we can use **standard SQL statements** on our dataframe. Therefore, we need to **register** our dataframe as a **temporary table**. Use the method **registerTempTable** on the bills_exploded dataframe and use the string 'bills_exploded' as the name of the registered table.

In [ ]:
# register table
bills_exploded.<FILL-IN>('bills_exploded')

Now you can use sql statements by using the sql method of the spark object. An example is shown here:

In [ ]:
# just execute
spark.sql("select * from bills_exploded").show()

Use the registered table or the domain specific spark sql syntax to answer the following questions:

**1. How many products have been sold?**  
**2. How many different products have been sold?**  
**3. Products of which category have been sold the most**  
**4. Which product has lead to the most total revenue?**  
**5. Which product is the cheapest and which is the most expensive one?**

Measure the execution time of the first and second statement. It is not that lightning fast, right? The reason is that the dataframe has not been cached. Please cache the dataframe by using the **cache()**. Call the result **bills_exploded_cached**. Afterwards, register again a temp table and answer all the questions. Do you notice a difference in speed for the second questions? Why do we not gain any speed boost for the first question?

In [ ]:
%%time
# 1.


In [ ]:
%%time
# 2.


In [ ]:
# Cache dataframe


In [ ]:
%%time
# 1.


In [ ]:
%%time
# 2.


In [ ]:
# 3.


In [ ]:
# 4. 


In [ ]:
# 5. 


For the next exercise we need the exploded dataframe. Hence, save the dataframe by using the method write.parquet method. As the argument use 'bills_exploded.parquet'.

In [ ]:
# save df
bills_exploded_cached.write.mode('overwrite').parquet(<FILL-IN>)

**This is the end of this exercise.**

Of course there are many more functions, like joins, etc. We will encounter some of them in the next exercise.